# MAP (programmation fonctionnelle en ARM)

Comment faire de la programmation fonctionnelle en ARM ?

Prenons l'exemple de map :

```
map(t_in,n,t_out,f) :
  pour i = 0..n-1 :
    t_out[i] = f(t_in[i])
```

Pour traduire la boucle, nous utiliserons le registre R6 :
```
init:    mov R6,#0
boucle:  cmp R6, n? @1
         beq fin
         @ corps de la boucle
         add R6, R6, #1
         b boucle 
fin:
```

Pour obtenir n et traduire le corps de la boucle il faut utiliser les paramètres de map. 

La méthode utilisée pour le passage des paramètres est une convention de passage par registres avec le premier paramètre dans R0, puis R1, etc.  :
* R0 pour t_in
* R1 pour n
* R2 pour t_2
* R3 pour f

Pour n (@1), on aura donc R1.

Pour l'accès à t[i], comme le tableau est un tableau d'entiers codés sur 1 octet, les adresses seront R0+R6 (ou R2+R6).

Pour l'appel à f, la convention d'appel utilise aussi les registres :
* R3 pour le paramètre
* R4 pour le résultat

Il y a un conflit d'utilisation pour R3 (adresse de f pour map, et paramètre de f en même temps). Pour résoudre ce conflit, on choisit de recopier l'adresse de f dans R7 (dans map).

Le corps de la boucle se traduit donc :
```
  @t_out[i] = f(t_in[i])
  ldr R3,[R0,R6]
  blx R7
  str R4,[R2,R6]
```

Le code complet (avec les prologues/épilogues nécessaires pour conserver les registres temporaires et les adresses de retour, nécessaire car appel imbriqué)

In [ ]:
%%writefile 
